In [ ]:
import json
import os
import numpy as np
import copy
import pandas as pd
from scenario_helper import rewrite_sentence

In [ ]:
temp = json.load(open("./scenarios/patients/gm.json"))

In [ ]:
available_scenarios = os.listdir("./scenarios/new_scenarios")

In [ ]:
available_scenarios

In [ ]:
available_scenarios = [available_scenarios[available_scenarios.index("eye_problems")]]

In [ ]:
s2ss={}

In [ ]:
for scenario in available_scenarios:
    print(scenario)
    scenario_path = os.path.join("./scenarios/new_scenarios", scenario)
    key_mapping = pd.read_csv(os.path.join(scenario_path, "key_mapping.csv"), quotechar='"')
    key_mapping.columns = key_mapping.columns.str.strip()
    key_responses = pd.read_csv(os.path.join(scenario_path, "key_responses.csv"), quotechar='"')
    mapping_dict = dict(zip(key_responses.iloc[:,0], key_responses.iloc[:,1]))
    for col in key_mapping.columns[1:]:
        key_mapping[col] = key_mapping[col].map(mapping_dict)
    key_mapping.columns = ["question"] + key_mapping.columns[1:].tolist()
    #########################
    baby_mapping = pd.read_csv(os.path.join(scenario_path, "baby_mapping.csv"), quotechar='"')
    baby_mapping.columns = baby_mapping.columns.str.strip()
    baby_responses = pd.read_csv(os.path.join(scenario_path, "baby_responses.csv"), quotechar='"')
    mapping_dict = dict(zip(baby_responses.iloc[:,0], baby_responses.iloc[:,1]))
    for col in baby_mapping.columns[1:]:
        baby_mapping[col] = baby_mapping[col].map(mapping_dict)
    baby_mapping.columns = ["question"] + baby_mapping.columns[1:].tolist()
    baby_qa = baby_mapping.set_index("question")
    ################
    mapping = pd.read_csv(os.path.join(scenario_path, "mapping.csv"), quotechar='"')
    mapping.columns = mapping.columns.str.strip()
    responses = pd.read_csv(os.path.join(scenario_path, "responses.csv"), quotechar='"')
    mapping_dict = dict(zip(responses.iloc[:,0], responses.iloc[:,1]))
    for col in mapping.columns[1:]:
        mapping[col] = mapping[col].map(mapping_dict)
    mapping.columns = ["question"] + mapping.columns[1:].tolist()
    ###################
    merged_df = pd.concat([mapping, key_mapping], axis=0, ignore_index=True)
    qa = merged_df.set_index("question")
    qa.columns = qa.columns.str.strip()
    meta = json.load(open(os.path.join(scenario_path, "meta.json")))
    task_name = meta["task_name"]
    problem = meta["problem"]
    subject = meta["subject"]
    causes = meta["illnesses"]
    relevant_characters = meta["relevant_characters"]
    characters = relevant_characters + ["others"]
    chars = meta["characters"]
    initial_state = [
            "interaction",
            f"{subject} have {problem}. can you help?"
        ]
    subjects = characters
    topics = list(set(list(mapping.iloc[:, 0].values) + list(key_mapping.iloc[:, 0].values)+ list(baby_mapping.iloc[:, 0].values)) )
    relevant_actions = []
    for q in list(key_mapping.iloc[:, 0].values):
        relevant_actions.append({
                "type": "interaction",
                "part": "discuss",
                "detail": f"customer,{q}",
                "sentence": f"i want to know about the customer 's {q}."
            })
    for q in ["Latching",
"Feeding Behavior",
"Oral Examination"]:
        relevant_actions.append({
                "type": "interaction",
                "part": "discuss",
                "detail": f"customer's child,{q}",
                "sentence": f"i want to know about the customer's child 's {q}."
            })
    posttest_actions = []
    for c in causes:
        posttest_actions.append({
                    "type": "posttest",
                    "part": "",
                    "detail": "",
                    "sentence": c
                })
    interaction = [{
                "type": "interaction",
                "part": "solution",
                "detail": "",
                "sentence": "i want to suggest a solution."
            }]
    for c in chars:
        for t in topics:
            interaction.append({
                "type": "interaction",
                "part": "discuss",
                "detail": f"{c},{t}",
                "sentence": f"i want to know about the {c} 's {t}."
            })
    temp["initial_state"] = initial_state
    temp["subjects"] = subjects
    temp["problem"] = problem
    temp["topics"] = topics
    temp["causes"] = causes
    temp["relevant_actions"] = relevant_actions 
    temp["posttest_actions"]["cause"] = posttest_actions
    temp["interaction"] = interaction
    json.dump(temp, open(os.path.join("./scenarios/patients", f"{task_name}.json"), "w"), indent=4)
    scenarios = [] 
    for c in qa.columns.tolist():
        print(c)
        s = copy.deepcopy(temp["scenarios"][0])
        s["question_answers"] = {}
        s["name"] = c
        s["posttest_as"]["cause"]=[{
                    "type": "posttest",
                    "part": "",
                    "detail": "",
                    "sentence": c
                }]
        for char in chars:
            if char not in s["question_answers"]:
                s["question_answers"][char] = {}
            for q in topics:
                if char not in relevant_characters and char != "customer's child":
                    s["question_answers"][char][q] = ["I don't understand how this is relevant."]
                elif char == "customer's child":
                    if q not in baby_qa.index:
                        s["question_answers"][char][q] = ["I don't understand how this is relevant."]
                    else:
                        answer = baby_qa.loc[q, c]
                        if not isinstance(answer,str):
                            answer = answer.values[0]
                        if answer not in s2ss.keys():
                            s2ss[answer] = rewrite_sentence(answer)
                        s["question_answers"][char][q] = s2ss[answer]
                else:
                    if q not in qa.index:
                        s["question_answers"][char][q] = ["I don't understand how this is relevant."]
                    else:
                        print(qa[c][q])
                        answer = qa[c][q]
                        if not isinstance(qa[c][q],str):
                            answer = qa[c][q].values[0]
                        if answer not in s2ss.keys():
                            s2ss[answer] = rewrite_sentence(answer)
                        s["question_answers"][char][q] = s2ss[answer]
                        print(q)
        scenarios.append(s)
    temp["scenarios"] = scenarios
    json.dump(temp, open(os.path.join("./scenarios/patients", f"{task_name}.json"), "w"), indent=4)


    #     





In [ ]:
for scenario in available_scenarios:
    print(scenario)
    scenario_path = os.path.join("./scenarios/new_scenarios", scenario)
    key_mapping = pd.read_csv(os.path.join(scenario_path, "key_mapping.csv"), quotechar='"')
    key_mapping.columns = key_mapping.columns.str.strip()
    key_responses = pd.read_csv(os.path.join(scenario_path, "key_responses.csv"), quotechar='"')
    mapping_dict = dict(zip(key_responses.iloc[:,0], key_responses.iloc[:,1]))
    for col in key_mapping.columns[1:]:
        key_mapping[col] = key_mapping[col].map(mapping_dict)
    key_mapping.columns = ["question"] + key_mapping.columns[1:].tolist()
    mapping = pd.read_csv(os.path.join(scenario_path, "mapping.csv"), quotechar='"')
    mapping.columns = mapping.columns.str.strip()
    responses = pd.read_csv(os.path.join(scenario_path, "responses.csv"), quotechar='"')
    mapping_dict = dict(zip(responses.iloc[:,0], responses.iloc[:,1]))
    
    for col in mapping.columns[1:]:
        mapping[col] = mapping[col].map(mapping_dict)
    mapping.columns = ["question"] + mapping.columns[1:].tolist()
    merged_df = pd.concat([mapping, key_mapping], axis=0, ignore_index=True)
    qa = merged_df.set_index("question")
    qa.columns = qa.columns.str.strip()
    meta = json.load(open(os.path.join(scenario_path, "meta.json")))
    task_name = meta["task_name"]
    problem = meta["problem"]
    subject = meta["subject"]
    causes = meta["illnesses"]
    relevant_characters = meta["relevant_characters"]
    characters = relevant_characters + ["others"]
    chars = meta["characters"]
    initial_state = [
            "interaction",
            f"{subject} have {problem}. can you help?"
        ]
    subjects = characters
    topics = list(set(list(mapping.iloc[:, 0].values) + list(key_mapping.iloc[:, 0].values)))
    relevant_actions = []
    for q in list(key_mapping.iloc[:, 0].values):
        relevant_actions.append({
                "type": "interaction",
                "part": "discuss",
                "detail": f"customer,{q}",
                "sentence": f"i want to know about the customer 's {q}."
            })
    posttest_actions = []
    for c in causes:
        posttest_actions.append({
                    "type": "posttest",
                    "part": "",
                    "detail": "",
                    "sentence": c
                })
    interaction = [{
                "type": "interaction",
                "part": "solution",
                "detail": "",
                "sentence": "i want to suggest a solution."
            }]
    for c in chars:
        for t in topics:
            interaction.append({
                "type": "interaction",
                "part": "discuss",
                "detail": f"{c},{t}",
                "sentence": f"i want to know about the {c} 's {t}."
            })
    temp["initial_state"] = initial_state
    temp["subjects"] = subjects
    temp["problem"] = problem
    temp["topics"] = topics
    temp["causes"] = causes
    temp["relevant_actions"] = relevant_actions 
    temp["posttest_actions"]["cause"] = posttest_actions
    temp["interaction"] = interaction
    json.dump(temp, open(os.path.join("./scenarios/patients", f"{task_name}.json"), "w"), indent=4)
    scenarios = [] 
    for c in qa.columns.tolist():
        print(c)
        s = copy.deepcopy(temp["scenarios"][0])
        s["question_answers"] = {}
        s["name"] = c
        s["posttest_as"]["cause"]=[{
                    "type": "posttest",
                    "part": "",
                    "detail": "",
                    "sentence": c
                }]
        for char in chars:
            if char not in s["question_answers"]:
                s["question_answers"][char] = {}
            for q in topics:
                if char not in relevant_characters:
                    s["question_answers"][char][q] = ["I don't understand how this is relevant."]
                else:
                    print(q)
                    if q not in qa.index:
                        s["question_answers"][char][q] = ["I don't understand how this is relevant."]
                    else:
                        print(qa[c][q])
                        answer = qa[c][q]
                        if not isinstance(qa[c][q],str):
                            answer = qa[c][q].values[0]
                        if answer not in s2ss.keys():
                            s2ss[answer] = rewrite_sentence(answer)
                        s["question_answers"][char][q] = s2ss[answer]
                        print(q)
        scenarios.append(s)
    temp["scenarios"] = scenarios
    json.dump(temp, open(os.path.join("./scenarios/patients", f"{task_name}.json"), "w"), indent=4)
   

    #     
        
        
    
    

In [ ]:
merged_df


In [ ]:
json.load(open(os.path.join("./scenarios/patients", f"{task_name}.json"))

In [ ]:
qa

In [ ]:
mapping